# MT3DMS Problem 9

The purpose of this section is to recreate the example problems that were first described in the 1999 MT3DMS report.  Other sources demonstrate that the MF6 result and MT3D result are comparable (identical?)

Ten example problems appear in the 1999 MT3DMS manual, starting on page 130. This notebook demonstrates example 9 from the list below:

- *Two-Dimensional Application Example*

:::{note}
Substantial rewrite, and some debugging to get the example to run on my computer.  A generic cut-and-paste from [https://modflow6-examples.readthedocs.io/en/master/_notebooks/ex-gwt-mt3dms-p09.html](https://modflow6-examples.readthedocs.io/en/master/_notebooks/ex-gwt-mt3dms-p09.html) is likely to be quite unsucessful.  I had to modify parts of the script for my machine (ie. to find the compiled binary).  And one object needed changing.  The example code at the link will probably work fine if the install is identical in terms of file paths as the GitHub source.  Herein I am mostly proving it is portable.
:::

:::{warning}
The next two code cells are specific to my machine, the reset wipes the lernel memory when it attempts to build the notebook for caching and subsequent typesetting.  The `warnings ignore` turns off a lot of deprecation warnings.  The online reference is funny enough to share

> You should just fix your code but just in case,

```
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
```

The script generates a lot of output that can be suppressed by a `;`  as presented here I left all this on to help me get things working.

:::

:::{warning}
I have this working but this section is incomplete. Remove this message when plotting is figured out!
:::


In [1]:
%reset -f

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Initial setup

Import dependencies, define the example name and workspace, and read settings from environment variables.

The next cell performs a forced reset to clear active kernel memory.  Ususlly unnecessary, but herein used to develop the example

In [3]:
import os
import pathlib as pl
from pprint import pformat

import flopy
import git
import matplotlib.pyplot as plt
import numpy as np
from flopy.plot.styles import styles
from modflow_devtools.misc import get_env, timed
import warnings
warnings.filterwarnings('ignore')

### Define parameters

Define model units, parameters and other settings.

In [4]:
# Model units
length_units = "meters"
time_units = "seconds"

# Model parameters
nlay = 1  # Number of layers
nrow = 18  # Number of rows
ncol = 14  # Number of columns
delr = 100.0  # Column width ($m$)
delc = 100.0  # Row width ($m$)
delz = 10.0  # Layer thickness ($m$)
top = 0.0  # Top of the model ($m$)
prsity = 0.3  # Porosity
k1 = 1.474e-4  # Horiz. hyd. conductivity of fine grain material ($m/sec$)
k2 = 1.474e-7  # Horiz. hyd. conductivity of medium grain material ($m/sec$)
inj = 0.001  # Injection well rate ($m^3/sec$)
ext = -0.0189  # Extraction well pumping rate ($m^3/sec$)
al = 20.0  # Longitudinal dispersivity ($m$)
trpt = 0.2  # Ratio of horiz. transverse to longitudinal dispersivity ($m$)
perlen = 2.0  # Simulation time ($years$)

# Additional model input
hk = k1 * np.ones((nlay, nrow, ncol), dtype=float)
hk[:, 5:8, 1:8] = k2
laytyp = icelltype = 0

# Active model domain
ibound = np.ones((nlay, nrow, ncol), dtype=int)
ibound[0, 0, :] = -1
ibound[0, -1, :] = -1
idomain = np.ones((nlay, nrow, ncol), dtype=int)
icbund = 1

# Boundary conditions
# MF2K5 pumping info
qwell1 = 0.001
qwell2 = -0.0189
welspd = {0: [[0, 3, 6, qwell1], [0, 10, 6, qwell2]]}  # Well pumping info for MF2K5
cwell1 = 57.87
cwell0 = 0.0
spd = {
    0: [[0, 3, 6, cwell1, 2], [0, 10, 6, cwell0, 2]],
    1: [[0, 3, 6, cwell0, 2], [0, 10, 6, cwell0, 2]],
}  # Well info 4 MT3D
# MF6 pumping information
wellist_sp1 = []
#                   (k,  i, j),   flow,  conc
wellist_sp1.append([(0, 3, 6), qwell1, cwell1])  # Injection well
wellist_sp1.append([(0, 10, 6), qwell2, cwell0])  # Pumping well
#
wellist_sp2 = []
#                   (k,  i, j),   flow,  conc
wellist_sp2.append([(0, 3, 6), qwell1, cwell0])  # Injection well
wellist_sp2.append([(0, 10, 6), qwell2, cwell0])  # Pumping well
spd_mf6 = {0: wellist_sp1, 1: wellist_sp2}

# Transport related
sconc = 0.0
ath1 = al * trpt
dmcoef = 0.0  # m^2/s
# Time variables
perlen = [365.0 * 86400, 365.0 * 86400]
steady = [False, False]
nper = len(perlen)
nstp = [365, 365]
tsmult = [1.0, 1.0]
#
sconc = 0.0
c0 = 0.0
botm = [top - delz]
mixelm = -1

# Solver settings
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0
percel = 1.0  # HMOC parameters
itrack = 2
wd = 0.5
dceps = 1.0e-5
nplane = 0
npl = 0
nph = 16
npmin = 2
npmax = 32
dchmoc = 1.0e-3
nlsink = nplane
npsink = nph
nadvfd = 1

## Model setup

Define functions to build models, write input files, and run the simulation.

**Groundwater Flow Package Build**

In [5]:
# Workspace and Executibles
binary = "/home/sensei/ce-4363-webroot/ModflowExperimental/mf6-arm/mf6"  # location on MY computer of the compiled modflow program
workarea = "/home/sensei/ce-4363-webroot/ModflowExperimental/mf6-arm/mt3d_example9" # location on MY computer to store files this example (directory must already exist)
#workarea = workspace # location on MY computer to store files this example (directory must already exist)

# Set Simulation Name(s)
name = "mt3d_ex09"
gwfname = "gwf-" + name
gwtname = "gwt-" + name

##### FLOPY Build simulation framework ####
sim = flopy.mf6.MFSimulation(
    sim_name="sim-" + name, exe_name=binary, version="mf6", sim_ws=workarea
)
####### CREATED "/home/sensei/ce-4363-webroot/ModflowExperimental/mf6-arm/mt3d_example9"

In [6]:
####### Instantiating MODFLOW 6 time discretization ########
tdis_rc = []
for i in range(nper):
    tdis_rc.append((perlen[i], nstp[i], tsmult[i]))
flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units);
## delete ";" in above line at end to show full output

In [7]:
####### Instantiating MODFLOW 6 groundwater flow model ########
# Set Model Name (using same base name as the simulation)
model_nam_file = "{}.nam".format(gwfname)
# create MODFLOW6 flow model framework
gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, save_flows=True, model_nam_file=model_nam_file)#;
## delete ";" in above line at end to show full output

In [8]:
###### Instantiating MODFLOW 6 solver for flow model #######
# Set Iterative Model Solution (choose solver parameters)
# about IMS see: https://water.usgs.gov/nrp/gwsoftware/ModelMuse/Help/sms_sparse_matrix_solution_pac.htm
# using defaults see: https://flopy.readthedocs.io/en/3.3.2/source/flopy.mf6.modflow.mfims.html
imsgwf = flopy.mf6.ModflowIms(
    sim,
    print_option="SUMMARY",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="NONE",
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=rclose,
    linear_acceleration="CG",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    filename=f"{gwfname}.ims",
    )
sim.register_ims_package(imsgwf, [gwf.name])#;
## delete ";" in above line at end to show full output

In [9]:
###### Instantiating MODFLOW 6 discretization package ######
flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
    idomain=idomain,
    filename=f"{gwfname}.dis",
    )#;
## delete ";" in above line at end to show full output

package_name = dis
filename = gwf-mt3d_ex09.dis
package_type = dis
model_or_simulation_package = model
model_name = gwf-mt3d_ex09

Block options
--------------------
length_units
{internal}
('meters')


Block dimensions
--------------------
nlay
{internal}
(1)

nrow
{internal}
(18)

ncol
{internal}
(14)


Block griddata
--------------------
delr
{constant 100.0}

delc
{constant 100.0}

top
{constant 0.0}

botm
{constant -10.0}

idomain
{internal}
(array([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1

In [10]:
###### Instantiating MODFLOW 6 initial conditions package for flow model #######
strt = np.zeros((nlay, nrow, ncol), dtype=float)
strt[0, 0, :] = 250.0
xc = gwf.modelgrid.xcellcenters
for j in range(ncol):
    strt[0, -1, j] = 20.0 + (xc[-1, j] - xc[-1, 0]) * 2.5 / 100
flopy.mf6.ModflowGwfic(gwf, strt=strt, filename=f"{gwfname}.ic")#;
## delete ";" in above line at end to show full output

package_name = ic
filename = gwf-mt3d_ex09.ic
package_type = ic
model_or_simulation_package = model
model_name = gwf-mt3d_ex09

Block griddata
--------------------
strt
{internal}
(array([[[250. , 250. , 250. , 250. , 250. , 250. , 250. , 250. , 250. ,
         250. , 250. , 250. , 250. , 250. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
           0. ,   0. ,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
           0. ,   0. ,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
           0. ,   0. ,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
           0. ,   0. ,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
           0. ,   0. ,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
           0. ,   0. ,   0. ,   0. ,   0. ],


In [11]:
# Instantiating MODFLOW 6 node-property flow package
flopy.mf6.ModflowGwfnpf(
    gwf,
    save_flows=False,
    icelltype=icelltype,
    k=hk,
    k33=hk,
    save_specific_discharge=True,
    filename=f"{gwfname}.npf",
    )#;
## delete ";" in above line at end to show full output

package_name = npf
filename = gwf-mt3d_ex09.npf
package_type = npf
model_or_simulation_package = model
model_name = gwf-mt3d_ex09

Block options
--------------------
save_flows
{internal}
(False)

save_specific_discharge
{internal}
(True)


Block griddata
--------------------
icelltype
{constant 0}

k
{internal}
(array([[[1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04],
        [1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04],
        [1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04],
        [1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04, 1.474e-04,
         1.

In [12]:
# Instantiate storage package
sto = flopy.mf6.ModflowGwfsto(gwf, ss=1.0e-05)#;
## delete ";" in above line at end to show full output

In [13]:
# Instantiating MODFLOW 6 constant head package
# MF6 constant head boundaries:
chdspd = []
# Loop through the top & bottom sides.
for j in np.arange(ncol):
        #               l,  r, c,  head, conc
    chdspd.append([(0, 0, j), 250.0, 0.0])  # Top boundary
    hd = 20.0 + (xc[-1, j] - xc[-1, 0]) * 2.5 / 100
    chdspd.append([(0, 17, j), hd, 0.0])  # Bottom boundary
chdspd = {0: chdspd}

flopy.mf6.ModflowGwfchd(
    gwf,
    maxbound=len(chdspd),
    stress_period_data=chdspd,
    save_flows=False,
    auxiliary="CONCENTRATION",
    pname="CHD-1",
    filename=f"{gwfname}.chd",
    )#;
## delete ";" in above line at end to show full output

package_name = chd-1
filename = gwf-mt3d_ex09.chd
package_type = chd
model_or_simulation_package = model
model_name = gwf-mt3d_ex09

Block options
--------------------
auxiliary
{internal}
(rec.array([('auxiliary', 'CONCENTRATION')],
          dtype=[('auxiliary_0', 'O'), ('auxiliary_1', 'O')]))

save_flows
{internal}
(False)


Block dimensions
--------------------
maxbound
{internal}
(1)


Block period
--------------------
stress_period_data
{internal}
(rec.array([((0, 0, 0), 250. , 0.), ((0, 17, 0),  20. , 0.),
           ((0, 0, 1), 250. , 0.), ((0, 17, 1),  22.5, 0.),
           ((0, 0, 2), 250. , 0.), ((0, 17, 2),  25. , 0.),
           ((0, 0, 3), 250. , 0.), ((0, 17, 3),  27.5, 0.),
           ((0, 0, 4), 250. , 0.), ((0, 17, 4),  30. , 0.),
           ((0, 0, 5), 250. , 0.), ((0, 17, 5),  32.5, 0.),
           ((0, 0, 6), 250. , 0.), ((0, 17, 6),  35. , 0.),
           ((0, 0, 7), 250. , 0.), ((0, 17, 7),  37.5, 0.),
           ((0, 0, 8), 250. , 0.), ((0, 17, 8),  40. , 0.),
 

In [14]:
# Instantiate the wel package
flopy.mf6.ModflowGwfwel(
    gwf,
    print_input=True,
    print_flows=True,
    stress_period_data=spd_mf6,
    save_flows=False,
    auxiliary="CONCENTRATION",
    pname="WEL-1",
    filename=f"{gwfname}.wel",
    )#;
## delete ";" in above line at end to show full output

package_name = wel-1
filename = gwf-mt3d_ex09.wel
package_type = wel
model_or_simulation_package = model
model_name = gwf-mt3d_ex09

Block options
--------------------
auxiliary
{internal}
(rec.array([('auxiliary', 'CONCENTRATION')],
          dtype=[('auxiliary_0', 'O'), ('auxiliary_1', 'O')]))

print_input
{internal}
(True)

print_flows
{internal}
(True)

save_flows
{internal}
(False)


Block period
--------------------
stress_period_data
{internal}
(rec.array([((0, 3, 6),  0.001 , 0.), ((0, 10, 6), -0.0189, 0.)],
          dtype=[('cellid', 'O'), ('q', '<f8'), ('CONCENTRATION', '<f8')]))



In [15]:
# Instantiating MODFLOW 6 output control package for flow model
flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwfname}.hds",
    budget_filerecord=f"{gwfname}.bud",
    headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )#;
## delete ";" in above line at end to show full output

package_name = oc
filename = gwf-mt3d_ex09.oc
package_type = oc
model_or_simulation_package = model
model_name = gwf-mt3d_ex09

Block options
--------------------
budget_filerecord
{internal}
(rec.array([('gwf-mt3d_ex09.bud',)],
          dtype=[('budgetfile', 'O')]))

head_filerecord
{internal}
(rec.array([('gwf-mt3d_ex09.hds',)],
          dtype=[('headfile', 'O')]))

headprintrecord
{internal}
(rec.array([('COLUMNS', 10, 'WIDTH', 15, 'DIGITS', 6, 'GENERAL')],
          dtype=[('columns_label', 'O'), ('columns', '<i8'), ('width_label', 'O'), ('width', '<i8'), ('digits_label', 'O'), ('digits', '<i8'), ('format', 'O')]))


Block period
--------------------
saverecord
{internal}
(rec.array([('HEAD', 'LAST', None), ('BUDGET', 'LAST', None)],
          dtype=[('rtype', 'O'), ('ocsetting', 'O'), ('ocsetting_data', 'O')]))

printrecord
{internal}
(rec.array([('HEAD', 'LAST', None), ('BUDGET', 'LAST', None)],
          dtype=[('rtype', 'O'), ('ocsetting', 'O'), ('ocsetting_data', 'O')]))





**Transport Package Building**

In [16]:
###### Instantiating MODFLOW 6 groundwater transport package ##########
gwtname = "gwt-" + name
gwt = flopy.mf6.MFModel(
    sim,
    model_type="gwt6",
    modelname=gwtname,
    model_nam_file=f"{gwtname}.nam",
    )#;
## delete ";" in above line at end to show full output
gwt.name_file.save_flows = True

In [17]:
# create iterative model solution and register the gwt model with it
imsgwt = flopy.mf6.ModflowIms(
    sim,
    print_option="SUMMARY",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="NONE",
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=rclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    filename=f"{gwtname}.ims",
    )
sim.register_ims_package(imsgwt, [gwt.name])#;
## delete ";" in above line at end to show full output

In [18]:
###### Instantiating MODFLOW 6 transport discretization package #####
flopy.mf6.ModflowGwtdis(
    gwt,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
    idomain=idomain,
    filename=f"{gwtname}.dis",
    )#;
## delete ";" in above line at end to show full output

package_name = dis
filename = gwt-mt3d_ex09.dis
package_type = dis
model_or_simulation_package = model
model_name = gwt-mt3d_ex09

Block dimensions
--------------------
nlay
{internal}
(1)

nrow
{internal}
(18)

ncol
{internal}
(14)


Block griddata
--------------------
delr
{constant 100.0}

delc
{constant 100.0}

top
{constant 0.0}

botm
{constant -10.0}

idomain
{internal}
(array([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [19]:
# Instantiating MODFLOW 6 transport initial concentrations
flopy.mf6.ModflowGwtic(gwt, strt=sconc, filename=f"{gwtname}.ic")#;
## delete ";" in above line at end to show full output

package_name = ic
filename = gwt-mt3d_ex09.ic
package_type = ic
model_or_simulation_package = model
model_name = gwt-mt3d_ex09

Block griddata
--------------------
strt
{constant 0.0}



In [20]:
    # Instantiating MODFLOW 6 transport advection package
    if mixelm >= 0:
        scheme = "UPSTREAM"
    elif mixelm == -1:
        scheme = "TVD"
    else:
        raise Exception()
    flopy.mf6.ModflowGwtadv(gwt, scheme=scheme, filename=f"{gwtname}.adv")#;
## delete ";" in above line at end to show full output

package_name = adv
filename = gwt-mt3d_ex09.adv
package_type = adv
model_or_simulation_package = model
model_name = gwt-mt3d_ex09

Block options
--------------------
scheme
{internal}
('tvd')



In [21]:
# Instantiating MODFLOW 6 transport dispersion package
if al != 0:
    flopy.mf6.ModflowGwtdsp(
        gwt,
        alh=al,
        ath1=ath1,
        filename=f"{gwtname}.dsp",
        )#;
## delete ";" in above line at end to show full output

In [22]:
# Instantiating MODFLOW 6 transport mass storage package
flopy.mf6.ModflowGwtmst(
    gwt,
    porosity=prsity,
    first_order_decay=False,
    decay=None,
    decay_sorbed=None,
    sorption=None,
    bulk_density=None,
    distcoef=None,
    filename=f"{gwtname}.mst",
    )#;
## delete ";" in above line at end to show full output

package_name = mst
filename = gwt-mt3d_ex09.mst
package_type = mst
model_or_simulation_package = model
model_name = gwt-mt3d_ex09

Block options
--------------------
first_order_decay
{internal}
(False)


Block griddata
--------------------
porosity
{constant 0.3}



In [23]:
# Instantiating MODFLOW 6 transport source-sink mixing package
sourcerecarray = [
    ("WEL-1", "AUX", "CONCENTRATION"),
    ("CHD-1", "AUX", "CONCENTRATION"),
    ]
flopy.mf6.ModflowGwtssm(
    gwt,
    sources=sourcerecarray,
    print_flows=True,
    filename=f"{gwtname}.ssm",
    )#;
## delete ";" in above line at end to show full output

package_name = ssm
filename = gwt-mt3d_ex09.ssm
package_type = ssm
model_or_simulation_package = model
model_name = gwt-mt3d_ex09

Block options
--------------------
print_flows
{internal}
(True)


Block sources
--------------------
sources
{internal}
(rec.array([('WEL-1', 'AUX', 'CONCENTRATION'),
           ('CHD-1', 'AUX', 'CONCENTRATION')],
          dtype=[('pname', 'O'), ('srctype', 'O'), ('auxname', 'O')]))



In [24]:
# Instantiating MODFLOW 6 transport output control package
flopy.mf6.ModflowGwtoc(
    gwt,
    budget_filerecord=f"{gwtname}.cbc",
    concentration_filerecord=f"{gwtname}.ucn",
    concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    saverecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")],
    printrecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")],
    filename=f"{gwtname}.oc",
    )

package_name = oc
filename = gwt-mt3d_ex09.oc
package_type = oc
model_or_simulation_package = model
model_name = gwt-mt3d_ex09

Block options
--------------------
budget_filerecord
{internal}
(rec.array([('gwt-mt3d_ex09.cbc',)],
          dtype=[('budgetfile', 'O')]))

concentration_filerecord
{internal}
(rec.array([('gwt-mt3d_ex09.ucn',)],
          dtype=[('concentrationfile', 'O')]))

concentrationprintrecord
{internal}
(rec.array([('COLUMNS', 10, 'WIDTH', 15, 'DIGITS', 6, 'GENERAL')],
          dtype=[('columns_label', 'O'), ('columns', '<i8'), ('width_label', 'O'), ('width', '<i8'), ('digits_label', 'O'), ('digits', '<i8'), ('format', 'O')]))


Block period
--------------------
saverecord
{internal}
(rec.array([('CONCENTRATION', 'LAST', None), ('BUDGET', 'LAST', None)],
          dtype=[('rtype', 'O'), ('ocsetting', 'O'), ('ocsetting_data', 'O')]))

printrecord
{internal}
(rec.array([('CONCENTRATION', 'LAST', None), ('BUDGET', 'LAST', None)],
          dtype=[('rtype', 'O'), ('ocs

In [25]:
# Instantiating MODFLOW 6 flow-transport exchange mechanism
flopy.mf6.ModflowGwfgwt(
    sim,
    exgtype="GWF6-GWT6",
    exgmnamea=gwfname,
    exgmnameb=gwtname,
    filename=f"{name}.gwfgwt",
    )

package_name = mt3d_ex09.gwfgwt
filename = mt3d_ex09.gwfgwt
package_type = gwfgwt
model_or_simulation_package = simulation
simulation_name = sim-mt3d_ex09


**Generate the Files**

In [26]:
sim.write_simulation(silent=True)

**Running the Model** 

In [27]:
success, buff = sim.run_simulation(silent=True, report=True)
assert success, pformat(buff)

### Plotting results

Define functions to plot model results.